In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
from discontinuitypy.datasets import IDsDataset
from beforerr.polars import pl_norm

from sunpy.time import TimeRange
from datetime import timedelta
from space_analysis.utils.speasy import Variables

09-Feb-24 16:50:17: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
    from . import skintegrate
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
    from sklearn.base import BaseEstimator
ModuleNotFoundError: No module named 'sklearn'


09-Feb-24 16:50:17: UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.

09-Feb-24 16:50:17: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 105, in <module>
    from . import nltk_stages
  File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/nltk_stages.py", line 19, in <module>
    import nltk
ModuleNotFoundError: No module named 'nltk'


09-Feb-2

In [3]:
import astropy.units as u
from astropy.constants import m_p

def thermal_spd2temp(speed, speed_unit=u.km / u.s):
    return (m_p * (speed * speed_unit) ** 2 / 2).to("eV").value


def df_thermal_spd2temp(ldf: pl.LazyFrame, speed_col, speed_unit=u.km / u.s):
    df = ldf.collect()
    return df.with_columns(
        plasma_temperature=thermal_spd2temp(df[speed_col].to_numpy(), speed_unit)
    ).lazy()

In [4]:
def get_and_process_data(
    mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, timerange, tau, ts
):
    # define variables
    mag_vars = Variables(
        dataset=mag_dataset,
        parameters=mag_parameters,
        timerange=timerange,
    ).retrieve_data()

    plasma_vars = Variables(
        dataset=plasma_dataset,
        parameters=plasma_parameters,
        timerange=timerange,
    ).retrieve_data()

    # get column names
    bcols = mag_vars.data[0].columns
    density_col = plasma_vars.data[0].columns[0]
    vec_cols = plasma_vars.data[1].columns
    temperature_col = plasma_vars.data[2].columns[0]

    # get data
    mag_data = mag_vars.to_polars()
    plasma_data = (
        plasma_vars.to_polars()
        .with_columns(plasma_speed=pl_norm(vec_cols))
        .rename({density_col: "plasma_density"})
    )
    # process temperature data
    if plasma_vars.data[2].unit == "km/s":
        plasma_data = plasma_data.pipe(df_thermal_spd2temp, temperature_col)
    else:
        plasma_data = plasma_data.rename({temperature_col: "plasma_temperature"})

    return IDsDataset(
        mag_data=mag_data,
        plasma_data=plasma_data,
        tau=tau,
        ts=ts,
        bcols=bcols,
        vec_cols=vec_cols,
        density_col="plasma_density",
        speed_col="plasma_speed",
        temperature_col="plasma_temperature",
    )

In [5]:
start = "2019-04-07T01:00"
end = "2019-04-07T12:00"

earth_start = "2019-04-09"
earth_end = "2019-04-12"

In [6]:
psp_timerange = TimeRange(start, end)
earth_timerange = TimeRange(earth_start, earth_end)

In [7]:
def validate(timerange):
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

In [8]:
tau = timedelta(seconds=16)
ts = timedelta(seconds=1 / 180)

mag_dataset = "PSP_FLD_L2_MAG_RTN"
mag_parameters = ["psp_fld_l2_mag_RTN"]

plasma_dataset = "PSP_SWP_SPI_SF00_L3_MOM"
plasma_parameters = ["DENS", "VEL_RTN_SUN", "TEMP", "SUN_DIST"]

timerange = validate(psp_timerange)

fname = "../data/psp_ids_dataset.arrow"

In [9]:
psp_ids_dataset = get_and_process_data(
    mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, timerange, tau, ts
).find_events().update_candidates_with_plasma_data().export(fname)

09-Feb-24 16:50:19: UserWarning: Ray execution environment not yet initialized. Initializing...
To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()


2024-02-09 16:50:22,086	INFO worker.py:1724 -- Started a local Ray instance.
09-Feb-24 16:50:23: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=58910) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=58910)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
(_deploy_ray_func pid=58910)     from . import skintegrate
(_deploy_ray_func pid=58910)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
(_deploy_ray_func pid=58910)     from sklearn.base import BaseEstimator
(_deploy_ray_func pid=58910) ModuleNotFoundError: No module named 'sklearn'
(_deploy_ray_func pid=58910) 
(_deploy_ray_func pid=58910) UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.
(_deploy_ray_func pid=58910) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=58910)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 105, in <module>

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

## Wind dataset


In [10]:
ts = timedelta(seconds=1 / 11)
tau = timedelta(seconds=30)

mag_dataset = "WI_H2_MFI"
# mag_parameters = ["BGSM"]
mag_parameters = ["BGSE"]

# plasma_dataset="WI_K0_SWE"
# plasma_parameters=["Np", "V_GSM", "THERMAL_SPD"]

plasma_dataset="WI_PM_3DP"
plasma_parameters=["P_DENS", "P_VELS", "P_TEMP"] # Proton velocity vector (GSE)

timerange = validate(earth_timerange)

fname = "../data/wind_ids_dataset_r.arrow"

In [11]:
wind_ids_dataset = get_and_process_data(
    mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, timerange, tau, ts
).find_events().update_candidates_with_plasma_data().export(fname)

09-Feb-24 16:50:28: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=58910) UserWarning: Distributing <class 'dict'> object. This may take some time.
Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
(_deploy_ray_func pid=58908) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=58908)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/__init__.py", line 85, in <module>
(_deploy_ray_func pid=58908)     from . import skintegrate
(_deploy_ray_func pid=58908)   File "/Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/pdpipe/skintegrate.py", line 20, in <module>
(_deploy_ray_func pid=58908)     from sklearn.base import BaseEstimator
(_deploy_ray_func pid=58908) ModuleNotFoundError: No module named 'sklearn'
(_deploy_ray_func pid=58908) 
(_deploy_ray_func pid=58908) UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.
(_deploy_ray_func pid=58908) UserWa

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [12]:
import hvplot.polars

In [16]:
wind_ids_dataset.plot()

2024-02-09 16:50:46.654 | INFO     | discontinuitypy.datasets:overview_plot:167 - Overview plot: 2019-04-11 16:45:00 - 2019-04-11 16:45:30
2024-02-09 16:50:46.654 | DEBUG    | discontinuitypy.datasets:log_event_change:79 - CHANGE INFO
        n.change: -0.0016632080078125
        v.ion.change: -0.07276110613753417
        T.change: 0.10945892333984375
        v.Alfven.change: 0.16010592925675837
        v.ion.change.l: -7.858156252181972
        v.Alfven.change.l: -15.310337071702092
        


:Layout
   .Overlay.I   :Overlay
      .NdOverlay.I :NdOverlay   [B comp]
         :Curve   [time]   (B)
      .VLine.I     :VLine   [x,y]
      .VLine.II    :VLine   [x,y]
   .Overlay.II  :Overlay
      .Curve.I   :Curve   [time]   (plasma_density)
      .Scatter.I :Scatter   [time]   (plasma_density)
      .VLine.I   :VLine   [x,y]
      .VLine.II  :VLine   [x,y]
   .Overlay.III :Overlay
      .NdOverlay.I :NdOverlay   [veloity comp]
         :Curve   [time]   (v)
      .VLine.I     :VLine   [x,y]
      .VLine.II    :VLine   [x,y]
   .Overlay.IV  :Overlay
      .Curve.I  :Curve   [time]   (plasma_temperature)
      .VLine.I  :VLine   [x,y]
      .VLine.II :VLine   [x,y]